In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
from dataloader import get_evaluation_datasets_by_client  # Assuming this function gets local client datasets
from model import Net
from collections import OrderedDict
from config import NUM_CLASSES, MODEL_PATH, BATCH_SIZE
from torch.utils.data import DataLoader
from utils import to_tensor
import pandas as pd
import pickle
import time

/home/sharedrive/PythonCodes/.venv311_new/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Helper Functions

In [2]:
# Load the global model from the saved path
def load_model(input_size, num_classes=NUM_CLASSES, model_path=MODEL_PATH):
    model = Net(input_size=input_size, num_classes=num_classes)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

In [3]:
# Run inference on a client's dataset
def run_inference(model, dataloader, device):
    all_preds = []
    all_labels = []

    # Get the total number of samples from the DataLoader
    total_samples = len(dataloader.dataset)
    # Start the timer before the loop
    start_time = time.time()
    
    with torch.no_grad():
        for batch in dataloader:
            features, labels = batch[0].to(device), batch[1].to(device)
            outputs = model(features)
            _, preds = torch.max(outputs.data, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # End the timer after the loop
    end_time = time.time()    
    # Calculate the total inference time
    total_inference_time = end_time - start_time    
    # Calculate average inference time per sample
    #inference_time_per_sample =  total_inference_time * 1000
    inference_time_per_sample =  total_inference_time * 1000000 / total_samples
    
    return np.array(all_preds), np.array(all_labels), f'{inference_time_per_sample:.4f} us'

In [4]:
def plot_confusion_matrix(y_true, y_pred, classes, title, ax):
    cm = confusion_matrix(y_true, y_pred, labels=classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    disp.plot(cmap=plt.cm.Blues, ax=ax)  # Pass ax here directly
    ax.set_title(title)  # Optional: Set a title for each subplot


## 2. Performance/History of Global Model

In [5]:
result_sources = {
    'components': [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
    'folds': [1, 2, 3, 4, 5],
    'marker': ['o', '-', '^' 'x', '-o-'],
    'clients': [1, 2, 3, 4],
    'path': './results/client_{0}/feature_{1}_fold_{2}_model.pth'
}


In [8]:
def plot_metrics(y_true, y_pred, classes, title, ax):
    cm = confusion_matrix(y_true, y_pred, labels=classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    disp.plot(cmap=plt.cm.Blues, ax=ax)  # Pass ax here directly
    ax.set_title(title)  # Optional

### 2.1 Accuracy/Loss vs Round

In [ ]:
# for component in result_sources.get('components'):    
#     loss_distributed = [] 
#     loss_centralized = [] 
#     accuracy_distributed =[] 
#     accuracy_centralized = []
   
#     for fold in result_sources.get('folds'):
#         history_path = result_sources.get('path').format(component, fold) + '/history.pkl'        
#         l_d, l_c, a_d, a_c = parse_history(history_path)        
#         loss_distributed.append(l_d)
#         loss_centralized.append(l_c)
#         accuracy_distributed.append(a_d)
#         accuracy_centralized.append(a_c)

#     history_plots = [
#         {
#             'type': 'distributed_loss',
#             'plot_name': 'Distributed Loss {}',
#             'x': 'Rounds',
#             'y': 'Loss',
#             'plot_position': [0, 0],
#             'data': loss_distributed,
#             'colors': ['red', 'brown', 'blue', 'purple', 'green']
#         },
#         {
#             'type': 'accuracy_distributed',
#             'plot_name': 'Distributed Accuracy {}',
#             'x': 'Rounds',
#             'y': 'Accuracy',
#             'plot_position': [0, 1],
#             'data': accuracy_distributed,
#             'colors': ['red', 'brown', 'blue', 'purple', 'green']
#         },
#          {
#             'type': 'centralized_loss',
#             'plot_name': 'Centralized Loss {}',
#             'x': 'Rounds',
#             'y': 'Loss',
#             'plot_position': [1, 0],
#             'data': loss_centralized,
#             'colors': ['red', 'brown', 'blue', 'purple', 'green']
#         },
#         {
#             'type': 'centralized_accuracy',
#             'plot_name': 'Centralized Accuracy {}',
#             'x': 'Rounds',
#             'y': 'Accuracy',
#             'plot_position': [1, 1],
#             'data': accuracy_centralized,
#             'colors': ['red', 'brown', 'blue', 'purple', 'green']
#         },     
      
#     ]

#     fig, ax = plt.subplots(2, 2, figsize=(16, 9))  # Adjust the figsize as needed
#     for plot in  history_plots:
#         position = plot.get('plot_position')
#         all_fold_data = plot.get('data')
#         #print(len(all_fold_data))
#         for i, data in enumerate(all_fold_data):
#             rounds = list(range(1, len(data)+1))
#             ax[position[0], position[1]].plot(rounds, data, label=f'Fold_{i+1}', marker='o', color=plot.get('colors')[i])
#             ax[position[0], position[1]].set_title(plot.get('plot_name').format(component))
#             ax[position[0], position[1]].set_xlabel(plot.get('x'))
#             ax[position[0], position[1]].set_ylabel(plot.get('y'))
#             ax[position[0], position[1]].legend()
#             ax[position[0], position[1]].grid(True)
            
            


### 2.2 Training Time

In [ ]:
# # ##Plotting the Time charts
# # for component in result_sources.get('components'):
# #     training_time = []
# #     for fold in result_sources.get('folds'):
# #         ##Parsign the training time
# #         training_time_path = result_sources.get('path').format(component, fold) + '/training_time.txt'
# #         training_time = parse_training_time(training_time_path)
# #         print(f'[Component {component} Fold {fold}]: {training_time} Seconds')

# for component in result_sources.get('components'):
#     training_time = []
#     for fold in result_sources.get('folds'):
#         ##Parsign the training time
#         training_time_path = result_sources.get('path').format(component, fold) + '/training_time.txt'
#         training_time.append(parse_training_time(training_time_path))
#         #print(f'[Component {component} Fold {fold}]: {training_time} Seconds')

#     training_time_to_string = ", ".join(map(str, training_time))
#     print(f'{component}, {training_time_to_string}')
   

## 3. Accumulate Results
- Accumulate all thre results and save in csv file
- It also stores values reauired for confusion matrix in a varialbe

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
client_metrics = {
    'Component': [],
    'Fold': [],
    'Client': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1_Score': [],
    'Sample_Number': [],
    'Inference_Time_Per_Sample': []
}

classes = np.arange(NUM_CLASSES)  # Define or import this variable

def accumulate_results(results, confusion_matrix_data):
    components = results.get('components')
    folds = results.get('folds')
    path = results.get('path')
    clients = results.get('clients')

    for client in clients:    
        for component in components:  
            for fold in folds:
                testset = get_evaluation_datasets_by_client(client, fold=fold, feature_count=component)  
                testloader = DataLoader(to_tensor(testset), batch_size=BATCH_SIZE)                
                
                model_path = path.format(client, component, fold)                        
                model = load_model(model_path=model_path, input_size=component, num_classes=NUM_CLASSES)
                model.to(device)

                preds, labels, inference_time_per_sample = run_inference(model, testloader, device)                                  
                   
                client_metrics['Component'].append(component)
                client_metrics['Fold'].append(fold)
                client_metrics['Client'].append(client)
                client_metrics['Accuracy'].append(accuracy_score(labels, preds))
                client_metrics['Precision'].append(precision_score(labels, preds))
                client_metrics['Recall'].append(recall_score(labels, preds))
                client_metrics['F1_Score'].append(f1_score(labels, preds))
                client_metrics['Sample_Number'].append(len(labels)),
                client_metrics['Inference_Time_Per_Sample'].append(inference_time_per_sample)

                #Saving info for confusion matrix
                key = f'{component}_{fold}_{client}'
                confusion_matrix_data[key] = {
                    'preds': preds,
                    'labels': labels,
                    'classes': np.arange(NUM_CLASSES)
                }   

    ##Converting into datafram for better visualization
    df = pd.DataFrame(client_metrics)
    print(df.to_string(index=False))
    return df, confusion_matrix_data           
    

In [12]:
confusion_matrix_data = {}
result_df, store_results_df = accumulate_results(result_sources, confusion_matrix_data)

 Component  Fold  Client  Accuracy  Precision   Recall  F1_Score  Sample_Number Inference_Time_Per_Sample
        12     1       1  0.591893   0.558900 1.000000  0.717044          84000                 6.6278 us
        12     2       1  0.673333   0.612845 1.000000  0.759955          84000                 5.4556 us
        12     3       1  0.806440   0.727635 0.999977  0.842340          84000                 5.4828 us
        12     4       1  0.839810   0.763478 1.000000  0.865877          84000                 5.4827 us
        12     5       1  0.718024   0.647130 0.999977  0.785760          84000                 5.4976 us
        13     1       1  0.731655   0.658351 1.000000  0.793982          84000                 6.7375 us
        13     2       1  0.862690   0.790176 1.000000  0.882792          84000                 5.4905 us
        13     3       1  0.574929   0.548832 0.999977  0.708698          84000                 5.5341 us
        13     4       1  0.621083   0.577102 

## 4. Confusion Matrix (per client per Fold)

In [13]:
plots = [
    {
        'client_id': 1,
        'plot_name': 'Client 1',
        'plot_position': [0, 0]
    },
    {
        'client_id': 2,
        'plot_name': 'Client 2',
        'plot_position': [0, 1]
    },
    {
        'client_id': 3,
        'plot_name': 'Client 3',
        'plot_position': [1, 0]
    },
    {
        'client_id': 4,
        'plot_name': 'Client 4',
        'plot_position': [1, 1]
    }   
]